<a href="https://colab.research.google.com/github/francescopatane96/Hidden_Markov_Models/blob/main/1Hidden_Markov_Models_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hidden Markov Models

HMM are probabilistic models used for structured predictions. they are popular for tagging all elements in a sequence (in the space or in the time) with some hidden states. in a hidden markov models there are 2 types of data: 

- States: ex. [cloud, rain, sun] or [in, out, transmembrane]. they are called 'hidden' because we don't see them.

- Observations: ex. [monday, tuesday, wednesday] or [A, Y, M] where A,Y,M are aminoacids incorporate in the protein sequence.

hidden states can be thought of as exstensions of so called Markov chains where the probability of the next hidden state is dependant on the current (t-1) hidden state, and the next observation is derived from that state. 

An example of this can be represented by speech tagging, where the observations are words and the hidden states are parts of speech.

'''Dynamic programming helps us when implementing an HMM. DP is a way of making your algorithm more efficient by storing some of the intermediate results. It's useful when your algorithm has a lot of repetitive computations.'''

HMM are composed by some elements:

1. transition probabilities matrix (matrix of size S by S
         stores probability from state i to state j)

2. emission probability matrix ( matrix of size S by N (number of observations)
         stores the probability of observing  O_j  from state  S_i)

3.  Initial state probabilities of size S. (A starting probability distribution over states)

4. Final probability: A final probability distribution over states

Another perspective of HMMs is that they are an extension on mixture models that includes a transition matrix. Conceptually, a mixture model has a set of "hidden" states---the mixture components---and one can calculate the probability that each sample belongs to each component. This approach treats each observations independently. However, like in the part-of-speech example we know that an adjective typically is followed by a noun, and so position in the sequence matters. A HMM adds a transition matrix between the hidden states to incorporate this information across the sequence, allowing for higher probabilities of transitioning from the "adjective" hidden state to a noun or verb.


pomegranate implements HMMs in a flexible manner

In [ ]:
pip install pomegranate

In [ ]:
pip install watermark

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy

from pomegranate import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

numpy      : 1.21.6
scipy      : 1.7.3
pomegranate: 0.14.8

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.10.133+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



CG islands identification example

- CG islands are CG rich regions on a sequence of DNA. 

DNA is made up of the four canonical nucleotides, abbreviated 'A', 'C', 'G', and 'T'. We can say that regions of the genome that are enriched for nucleotides 'C' and 'G' are 'CG islands'.

important: The issue with identifying these regions is that they are not exclusively made up of the nucleotides 'C' and 'G', but have some 'A's and 'T's scatted amongst them.

A simple model that looked for long stretches of C's and G's would not perform well, because it would miss most of the real regions.

We can start off by building the model. Because HMMs involve the transition matrix, which is often represented using a graph over the hidden states, building them requires a few more steps that a simple distribution or the mixture model. Our simple model will be composed of two distributions. One distribution wil be a uniform distribution across all four characters and one will have a preference for the nucleotides C and G, while still allowing the nucleotides A and T to be present.

In [ ]:
d1 = DiscreteDistribution({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})
d2 = DiscreteDistribution({'A': 0.10, 'C': 0.40, 'G': 0.40, 'T': 0.10})

For the HMM we have to first define a set of hidden states, which are a pair of a distribution and a name.

In [ ]:
s1 = State(d1, name='background')
s2 = State(d2, name='CG island')

Now we define the HMM and pass in the states.

In [ ]:
model = HiddenMarkovModel()
model.add_states(s1, s2)

Then we have to define the transition matrix, which is the probability of going from one hidden state to the next hidden state. In some cases, like this one, there are high self-loop probabilities, indicating that it's likely that one will stay in the same hidden state from one observation to the next in the sequence. Other cases have a lower probability of staying in the same state, like the part of speech tagger. A part of the transition matrix is the start probabilities, which is the probability of starting in each of the hidden states. Because we create these transitions one at a time, they are very amenable to sparse transition matrices, where it is impossible to transition from one hidden state to the next.

In [ ]:
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.9)
model.add_transition(s1, s2, 0.1)
model.add_transition(s2, s1, 0.1)
model.add_transition(s2, s2, 0.9)

Now, finally, we need to bake the model in order to finalize the internal structure. Bake must be called when the model has been fully specified.

In [ ]:
model.bake()

Now we can make predictions on some sequence. Let's create some sequence that has a CG enriched region in the middle and see whether we can identify it.

In [ ]:
seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111110000


It looks like it successfully identified a CG island in the middle (the long stretch of 0's) and another shorter one at the end. The predicted integers don't correspond to the order in which states were added to the model, but rather, the order that they exist in the model after a topological sort. More importantly, the model wasn't tricked into thinking that every CG or even pair of CGs was an island. It required many C's and G's to be part of a longer stretch to identify that region as an island. Naturally, the balance of the transition and emission probabilities will heavily influence what regions are detected.

Let's say, though, that we want to get rid of that CG island prediction at the end because we don't believe that real islands can occur at the end of the sequence. We can take care of this by adding in an explicit end state that only the non-island hidden state can get to. We enforce that the model has to end in the end state, and if only the non-island state gets there, the sequence of hidden states must end in the non-island state. Here's how:

In [ ]:
model = HiddenMarkovModel()
model.add_states(s1, s2)
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.89 )
model.add_transition(s1, s2, 0.10 )
model.add_transition(s1, model.end, 0.01)
model.add_transition(s2, s1, 0.1 )
model.add_transition(s2, s2, 0.9)
model.bake()

Note that all we did was add a transition from s1 to model.end with some low probability. This probability doesn't have to be high if there's only a single transition there, because there's no other possible way of getting to the end state.

In [ ]:
seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111111111


This seems far more reasonable. There is a single CG island surrounded by background sequence, and something at the end. If we knew that CG islands cannot occur at the end of sequences, we need only modify the underlying structure of the HMM in order to say that the sequence must end from the background state.

In the same way that mixtures could provide probabilistic estimates of class assignments rather than only hard labels, hidden Markov models can do the same. These estimates are the posterior probabilities of belonging to each of the hidden states given the observation, but also given the rest of the sequence.

In [ ]:
print(model.predict_proba(seq))

[[0.4827054  0.5172946 ]
 [0.42204064 0.57795936]
 [0.28598775 0.71401225]
 [0.25756456 0.74243544]
 [0.16040038 0.83959962]
 [0.13871983 0.86128017]
 [0.17217854 0.82782146]
 [0.14750165 0.85249835]
 [0.18021186 0.81978814]
 [0.15446181 0.84553819]
 [0.18788044 0.81211956]
 [0.16252887 0.83747113]
 [0.19841088 0.80158912]
 [0.32919701 0.67080299]
 [0.38366073 0.61633927]
 [0.58044619 0.41955381]
 [0.69075524 0.30924476]
 [0.74653183 0.25346817]
 [0.76392808 0.23607192]
 [0.7479817  0.2520183 ]
 [0.69407484 0.30592516]
 [0.74761829 0.25238171]
 [0.76321595 0.23678405]
 [0.74538467 0.25461533]
 [0.68896078 0.31103922]
 [0.57760471 0.42239529]
 [0.58391467 0.41608533]
 [0.53953778 0.46046222]
 [0.6030831  0.3969169 ]
 [0.61516689 0.38483311]
 [0.57928847 0.42071153]
 [0.48505793 0.51494207]
 [0.30518744 0.69481256]
 [0.25379428 0.74620572]
 [0.12610747 0.87389253]
 [0.08105965 0.91894035]
 [0.07637934 0.92362066]
 [0.10767468 0.89232532]
 [0.20431225 0.79568775]
 [0.23273876 0.76726124]


We can see here the transition from the first non-island region to the middle island region, with high probabilities in one column turning into high probabilities in the other column. The predict method is just taking the most likely element, the maximum-a-posteriori estimate.

In addition to using the forward-backward algorithm to just calculate posterior probabilities for each observation, we can count the number of transitions that are predicted to occur between the hidden states.

In [ ]:
trans, ems = model.forward_backward(seq)
print(trans)

[[15.78100555  2.89559314  0.          0.        ]
 [ 2.41288774 28.91051356  0.          1.        ]
 [ 0.4827054   0.5172946   0.          0.        ]
 [ 0.          0.          0.          0.        ]]


This is the transition table, which has the soft count of the number of transitions across an edge in the model given a single sequence. It is a square matrix of size equal to the number of states (including start and end state), with number of transitions from (row_id) to (column_id). This is exemplified by the 1.0 in the first row, indicating that there is one transition from background state to the end state, as that's the only way to reach the end state. However, the third (or fourth, depending on ordering) row is the transitions from the start state, and it only slightly favors the background state. These counts are not normalized to the length of the input sequence, but can easily be done so by dividing by row sums, column sums, or entire table sums, depending on your application.

A possible reason not to normalize is to run several sequences through and add up their tables, because normalizing in the end and extracting some domain knowledge. It is extremely useful in practice. For example, we can see that there is an expectation of ~2.9 transitions from CG island to background, and ~2.4 from background to CG island. This could be used to infer that there are ~2-3 edges, which makes sense if you consider that the start and end of the sequence seem like they might be part of the CG island states except for the strict transition probabilities used (look at the first few rows of the emission table above.)

In [ ]:
print(ems)

[[-0.72834874 -0.65914274]
 [-0.86265366 -0.54825173]
 [-1.25180628 -0.33685517]
 [-1.35648488 -0.29781936]
 [-1.83008222 -0.17483014]
 [-1.97529901 -0.14933542]
 [-1.75922331 -0.18895778]
 [-1.91391594 -0.159584  ]
 [-1.71362211 -0.19870934]
 [-1.86780839 -0.16778195]
 [-1.6719495  -0.2081077 ]
 [-1.81689961 -0.17736849]
 [-1.61741525 -0.22115912]
 [-1.11109888 -0.3992798 ]
 [-0.95799663 -0.48395771]
 [-0.54395817 -0.8685635 ]
 [-0.36996973 -1.17362222]
 [-0.29231702 -1.37251703]
 [-0.26928163 -1.44361878]
 [-0.29037676 -1.37825358]
 [-0.36517548 -1.18441479]
 [-0.29086274 -1.37681261]
 [-0.27021427 -1.44060672]
 [-0.29385485 -1.36800139]
 [-0.37257093 -1.16783627]
 [-0.54886554 -0.8618137 ]
 [-0.53800042 -0.87686492]
 [-0.61704247 -0.77552447]
 [-0.50570029 -0.92402833]
 [-0.48586168 -0.95494553]
 [-0.54595471 -0.86580787]
 [-0.72348695 -0.66370087]
 [-1.18682912 -0.36411317]
 [-1.37123124 -0.29275396]
 [-2.07062079 -0.13479788]
 [-2.51256996 -0.08453407]
 [-2.57204302 -0.07945383]
 

emission matrix